In [1]:
import torch
import pandas as pd
import numpy as np
import sentencepiece
from transformers import MT5Model, T5Tokenizer, MT5ForConditionalGeneration

In [3]:
d = pd.read_csv("scb-mt-en-th-2020/apdf.csv")
d.head()

,en_text,th_text
0,FAR LEFT: Indonesian National Police Chief Tit...,(ซ้ายสุด) นายติโต คาร์นาเวียน ผู้บัญชาการตำรวจ...
1,(Pictured: A rocket believed to be a Hwasong m...,(ภาพ: จรวดซึ่งเชื่อว่าเป็นขีปนาวุธฮวาซองเช่นเด...
2,"(Pictured: From left, Indian Air Commander Ak ...",(ภาพ: จากซ้าย นายอัค โชราเชีย ผู้บัญชาการทางอา...
3,"(Pictured: Left to right, Australia’s Defence ...",(ภาพ: จากซ้ายไปขวา นางมารีส เพย์น รัฐมนตรีว่าก...
4,(Pictured: A man watches the live-stream trans...,(ภาพ: ชายคนหนึ่งชมการแพร่ภาพออกอากาศสดการปล่อย...


In [4]:
testdata = d.head(1)

In [6]:
testdata["th_text"][0]

'(ซ้ายสุด) นายติโต คาร์นาเวียน ผู้บัญชาการตำรวจแห่งชาติอินโดนีเซีย (จากซ้าย) นายโรนัลด์ เดลา โรซา ผู้บัญชาการตำรวจแห่งชาติฟิลิปปินส์ และนายคาลิด อาบู บาการ์ ผู้บัญชาการตํารวจแห่งชาติมาเลเซีย ไขว้แขนกันก่อนเริ่มการประชุมความมั่นคงไตรภาคีในเมืองปาเซย์ ซึ่งอยู่ทางตะวันออกเฉียงใต้ของกรุงมะนิลา ประเทศฟิลิปปินส์ ในเดือนมิถุนายน พ.ศ. 2560 ดิแอสโซซิเอทเต็ด เพรส'

## Attempt with MT5 for Conditional Gen, following huggingface example

In [7]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")


In [8]:
en_test = testdata["en_text"][0]
th_test = testdata["th_text"][0]
inputs = tokenizer(en_test, return_tensors="pt")
inputs

{'input_ids': tensor([[ 94523,    259, 105188,    267,   3664,    272,   5139,  34330,    259,
          19421,    259, 149603, 141251, 119416,    261,    702,  12255,    261,
          67548,   5139,  34330,    259,  19421,  73418, 120219,  14537,    305,
          11061,   9277,    272,  34330,    336,  92559,   4724,  81879,    285,
          13222,  16900,    372,   5275,    259,  75807,   5038,    287,   1830,
         148512,  19004,  59905,    281,  39696,    276,   9416,    261,  48649,
          82365,    304,    259,  76931,    261,  56417,    261,    281,   1941,
          11141,    491,  47072,    259, 164956,  39761, 141163,    439,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
with tokenizer.as_target_tokenizer():
    labels = tokenizer(th_test, return_tensors="pt")

outputs = model(**inputs,labels=labels["input_ids"])
loss = outputs.loss

In [10]:
outputs

Seq2SeqLMOutput(loss=tensor(28.8294, grad_fn=<NllLossBackward>), logits=tensor([[[-80.3485, -39.7267, -78.1208,  ..., -75.5055, -79.4977, -79.8178],
         [-13.8893,  27.6761, -14.9983,  ..., -15.7269, -12.7509, -16.0421],
         [-81.4027,  -6.4177, -79.9628,  ..., -76.9944, -80.2652, -82.3468],
         ...,
         [-81.1343,  -6.0733, -79.7248,  ..., -76.5920, -80.1284, -82.0216],
         [-84.6230,  -3.0711, -83.1171,  ..., -79.7933, -83.6685, -85.6253],
         [-81.1102, -10.0905, -79.6209,  ..., -76.6086, -80.1649, -81.7401]]],
       grad_fn=<UnsafeViewBackward>), past_key_values=((tensor([[[[-2.2790e-01, -1.7210e-01,  5.3164e-01,  ..., -4.0905e-01,
           -2.1097e-01,  2.6987e-01],
          [ 1.5307e-01,  2.0417e-01, -2.1549e-01,  ..., -6.9238e-01,
            1.4608e+00, -4.1197e-01],
          [ 7.3765e-01, -2.6691e-01,  7.0493e-01,  ..., -1.7522e+00,
            5.5423e-01, -6.1998e-01],
          ...,
          [-1.8634e-01, -9.6978e-01, -4.5617e-01,  ...,  2

In [11]:
loss

tensor(28.8294, grad_fn=<NllLossBackward>)

In [12]:
tokenizer.decode(outputs)

ValueError: invalid literal for int() with base 10: 'loss'

In [14]:
input_ids = torch.tensor(tokenizer.encode(testdata["en_text"][0], add_special_tokens=True)).unsqueeze(0)

In [15]:
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

AttributeError: 'MT5Config' object has no attribute 'decoder'

In [16]:
tokenizer.decode(generated[0])

NameError: name 'generated' is not defined

In [ ]:
model = TFMT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
inputs = tokenizer('translate English to German: The house is wonderful.', return_tensors='tf').input_ids
labels = tokenizer('Das Haus ist wunderbar.', return_tensors='tf').input_ids

In [ ]:
outputs = model(inputs, labels=labels)

In [ ]:
loss = outputs.loss
loss

In [ ]:
logits = outputs.logits
logits

In [ ]:
inputs = tokenizer("summarize: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
inputs = tokenizer("translate English to Spanish: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
result = model.generate(inputs)

In [ ]:
result

In [ ]:
tokenizer.decode(result[0])